---
title: "Analiza Przeżycia"
subtitle: "Raport 2"
authors: "Jakub Zdancewicz, Ksawery Józefowski"
date: last-modified
date-format: "[Wygenerowano] D [października] YYYY"
format: 
  pdf:
    toc: true          
    toc-depth: 3       
    number-sections: true
crossref:  # recznie zmieniamy tytuly tabel itp. na polskie nazwy
  fig-title: "Rysunek"
  tbl-title: "Tabela"
  toc: true
  # eq-title: "Równanie"
  # lof-title: "Spis rysunków"
  # lot-title: "Spis tabel"
  # lst-title: "Listing"
# lang: pl  # jezyk polski psuje "cross references" oraz formatowanie w tabeli
# toc: true  # table of contents
execute:
  echo: true
---

# Wstęp
Sprawozdanie jest podzielone na cztery części.  

Część pierwsza obejmuje implementację i wizualizację estymatorów Kaplana-Meiera oraz Fleminga-Harringtona funkcji przeżycia czasu do remisji choroby u pacjentów leczonych lekami A i B. Dodatkowo oszacujemy ogon funkcji przeżycia według propozycji Browna, Hollandera i Kowara, dopasowując odpowiedni model wykładniczy. Na zakończenie, na 1000 wygenerowanych zbiorach danych, wyznaczymy estymacje wartości funkcji przeżycia w wybranych punktach korzystając z estymatora Kaplana-Meiera z oszacowanym ogonem oraz przedstawimy histogramy tych estymacji.

W drugiej części zajmiemy się implementacją estymatorów średniego czasu życia, opartych na estymatorach Kaplana-Meiera oraz Fleminga-Harringtona (z opcją szacowania ogona). Następnie wykorzystamy te estymatory do oszacowania średnich czasów do remisji choroby u pacjentów leczonych lekiem A oraz lekiem B.

W części trzeciej skupimy się na metodzie wyznaczania przedziałów ufności dla średniego czasu życia na zadanym poziomie ufności. Następnie zastosujemy ją do określenia przedziałów ufności dla średniego czasu do progresji raka jajnika w dwóch badanych grupach pacjentek Mayo Clinic.

Na końcu pochylimy się nad problemem weryfikacji hipotezy o jednakowym rozkładzie czasów życia w kilku podpopulacjach. W szczególności zastosujemy testy log-rank, Gehana-Breslowa, Tarone'a-Warego oraz Peto-Peto do zweryfikowania hipotezy o jednakowym rozkładzie czasu do progresji choroby w dwóch badanych grupach pacjentek z poprzedniego punktu. Przeanalizujemy wyniki testów, ze szczególnym uwzględnieniem otrzymanych $p$-wartości, oraz zastanowimy się nad ich związkiem z funkcjami wagowymi oraz estymatorami funkcji przeżycia.

In [1]:
#| echo: False
import numpy as np
import pandas as pd
import re
import math
import matplotlib.pyplot as plt
import scipy
from lifelines.statistics import multivariate_logrank_test
from scipy.stats import expon, gamma, binomtest, norm

from IPython.display import Markdown
from tabulate import tabulate

# Estymatory funkcji przeżycia

In [2]:
#| echo: False
# Grupa A
full_A = np.array([0.03345514, 0.08656403, 0.08799947, 0.24385821, 0.27755032,
                   0.40787247, 0.58825664, 0.64125620, 0.90679161, 0.94222208])
censored_A = np.ones(10)  # wartości dla danych cenzurowanych
data_A = np.concatenate([full_A, censored_A])
delta_A = np.array([True]*len(full_A) + [False]*len(censored_A))

# Grupa B
full_B = np.array([0.03788958, 0.12207257, 0.20319983, 0.24474299, 0.30492413,
                   0.34224462, 0.42950144, 0.44484582, 0.63805066, 0.69119721])
censored_B = np.ones(10)  # wartości dla danych cenzurowanych
data_B = np.concatenate([full_B, censored_B])
delta_B = np.array([True]*len(full_B) + [False]*len(censored_B))

df_A = pd.DataFrame({"Data": data_A, "Delta": delta_A})
df_B = pd.DataFrame({"Data": data_B, "Delta": delta_B})

Niech $X_1, \ldots, X_n$ będą niezależnymi zmiennymi losowymi o jednakowym, lecz nieznanym rozkładzie o dystrybuancie $F$. Zakładamy, że mamy do czynienia z danymi cenzurowanymi losowo. Z każdą zmienną $X_i$ związana jest zatem zmienna losowa cenzurująca $C_i$, niezależna od $X_i$.

Obserwujemy więc zmienne
$$
T_i = \min(X_i, C_i)
$$
oraz
$$
\delta_i =
\begin{cases}
1, & \text{gdy } X_i \le C_i,\\[4pt]
0, & \text{gdy } X_i > C_i 
\end{cases}
$$

Naszym celem jest estymacja funkcji przeżycia, zdefiniowanej jako
$$
S(t) = \mathbb{P}(X > t)
$$

Estymację przeprowadzimy na podstawie danych dotyczących czasu remisji choroby pacjentów leczonych lekami A i B.

## Estymator Kaplana-Meiera

Niech $t_{(1)} \le \ldots \le t_{(n)}$ będą uporządkowanymi rosnąco wartościami wektora losowego $(T_1, \ldots, T_n)$.
Ponadto niech $\delta_{(i)}$, dla $i = 1, \ldots, n$, oznacza wartość $\delta$ odpowiadającą obserwacji $t_{(i)}$.

Wówczas estymator Kaplana-Meiera definiujemy jako
$$
\hat{S}(t) =
\prod_{i :\, t_{(i)} \le t}
\left(\frac{r_i - d_i}{r_i} \right),
\qquad 0 \le t \le t_{(n)},
$$
gdzie
$$
r_i = \sum_{j=1}^n \mathbf{1}_{[\, t_{(i)},\infty)}(t_j)
\qquad \text{oraz} \qquad
d_i = \sum_{j=1}^n \mathbf{1}_{\{ t_{(i)} \}}(t_j)\, \mathbf{1}_{\{1\}}(\delta_j)
$$

Wielkość $r_i$ oznacza liczbę jednostek pozostających w stanie ryzyka w chwili $t_{(i)}$, natomiast $d_i$ jest liczbą zdarzeń (nieocenzurowanych), które wystąpiły dokładnie w punkcie $t_{(i)}$.

Poniżej przedstawiamy implementację tego estymatora w języku Python.

In [3]:
def S_hat(t, df):
    s_hat = 1
    data = df['Data'].to_numpy()
    delta = df['Delta'].to_numpy()
    order = data.argsort()
    data = data[order]
    delta = delta[order]

    unique_times = np.unique(data[data <= t])

    for s_i in unique_times:
        if s_i > t:
            break
        r_i = np.sum(data >= s_i)
        d_i = np.sum((data == s_i) & (delta == 1))
        if d_i == 0:
            continue
        s_hat *= (r_i - d_i) / r_i
    return s_hat

Na wykresach [-@fig-kmA] i [-@fig-kmB] przedstawiamy estymatory funkcji przeżycia czasu do remisji choroby u pacjentów leczonych lekami A i B, odpowiednio. Widzimy wyraźne różnice między wykresami estymatora, co pozwala przypuszczać, że leki A i B nie wykazują podobnego działania.

In [4]:
#| fig-cap: "Wykres estymatorów Kaplana-Meiera funkcji przeżycia czasu do remisji choroby u pacjentów leczonych lekiem A"
#| fig-align: center
#| label: kmA
#| echo: false
time = np.linspace(0, 1, 101)
prob = [S_hat(t, df_A) for t in time]
plt.figure(figsize=(5, 3))
plt.step(time, prob, where="post")
plt.ylim(0, 1.05)
plt.xlabel(r"Czas $t$")
plt.ylabel(r"Estymowana funkcja przeżycia $\hat{S}(t)$")
plt.title("Estymator Kaplana-Meiera dla leku A")
plt.tight_layout()
plt.savefig('km_A.pdf')
plt.close()

![Wykres estymatorów Kaplana-Meiera funkcji przeżycia czasu do remisji choroby u pacjentów leczonych lekiem A](km_A.pdf){#fig-kmA}

In [5]:
#| fig-cap: "Wykres estymatorów Kaplana-Meiera funkcji przeżycia czasu do remisji choroby u pacjentów leczonych lekiem B"
#| fig-align: center
#| label: kmB
#| echo: false
time = np.linspace(0, 1, 101)
prob = [S_hat(t, df_B) for t in time]
plt.figure(figsize=(5, 3))
plt.step(time, prob, where="post")
plt.ylim(0, 1.05)
plt.xlabel(r"Czas $t$")
plt.ylabel(r"Estymowana funkcja przeżycia $\hat{S}(t)$")
plt.title("Estymator Kaplana-Meiera dla leku B")
plt.tight_layout()
plt.savefig('km_B.pdf')
plt.close()

![Wykres estymatorów Kaplana-Meiera funkcji przeżycia czasu do remisji choroby u pacjentów leczonych lekiem B](km_B.pdf){#fig-kmB}

## Estymator Fleminga-Harringtona

Innym popularnym estymatorem funkcji przeżycia jest estymator Fleminga-Harringtona. Wykorzystuje on zależność między funkcją przeżycia a funkcją hazardu $h$:

$$
S(t) = \exp[-H(t)],
$$

gdzie

$$
H(t) = \int_0^t h(u)\, du,
$$

oraz estymator skumulowanej funkcji hazardu zaproponowany przez Nelsona-Aalena:

$$
\hat{H}(t) = \sum_{j:\, t_j \le t} \frac{d_j}{r_j}, \quad \text{dla } 0 \le t \le t_{(n)},
$$

gdzie $r_j$ i $d_j$ są zdefiniowane tak samo jak w estymatorze Kaplana-Meiera.

Estymator Fleminga-Harringtona jest więc dany wzorem:

$$
\hat{S}(t) = \exp\Biggl(- \sum_{j:\, t_j \le t} \frac{d_j}{r_j} \Biggr), \quad \text{dla } 0 \le t \le t_{(n)}.
$$

Poniżej przedstawiamy implementację tego estymatora w języku Python.


In [6]:
def S_tld(t, df):
    data = df['Data'].to_numpy()
    delta = df['Delta'].to_numpy()
    order = data.argsort()
    data = data[order]
    delta = delta[order]
    s_tld = 0.0    
    for j in range(len(data)):
        if data[j] > t:
            break
        r_j = np.sum(data >= data[j])
        d_j = np.sum((data == data[j]) & (delta == 1))
        s_tld += d_j / r_j
    return np.exp(-s_tld)

In [7]:
#| fig-cap: "Wykres estymatorów Fleminga-Harringtona funkcji przeżycia czasu do remisji choroby u pacjentów leczonych lekiem A"
#| fig-align: center
#| label: fh-A
#| echo: false
time = np.linspace(0, 1, 101)
prob = [S_tld(t, df_A) for t in time]
plt.figure(figsize=(5, 3))
plt.step(time, prob, where="post")
plt.ylim(0, 1.05)
plt.xlabel(r"Czas $t$")
plt.ylabel(r"Estymowana funkcja przeżycia $\hat{S}(t)$")
plt.title("Estymator Fleminga-Harringtona dla leku A")
plt.tight_layout()
plt.savefig('fh_A.pdf')
plt.close()

![Wykres estymatorów Fleminga-Harringtona funkcji przeżycia czasu do remisji choroby u pacjentów leczonych lekiem A](fh_A.pdf){#fig-fhA}

In [8]:
#| fig-cap: "Wykres estymatorów Fleminga-Harringtona funkcji przeżycia czasu do remisji choroby u pacjentów leczonych lekiem B"
#| fig-align: center
#| label: fh-B
#| echo: false
time = np.linspace(0, 1, 101)
prob = [S_tld(t, df_B) for t in time]
plt.figure(figsize=(5, 3))
plt.step(time, prob, where="post")
plt.ylim(0, 1.05)
plt.xlabel(r"Czas $t$")
plt.ylabel(r"Estymowana funkcja przeżycia $\hat{S}(t)$")
plt.title("Estymator Fleminga-Harringtona dla leku B")
plt.tight_layout()
plt.savefig('fh_B.pdf')
plt.close()

![Wykres estymatorów Fleminga-Harringtona funkcji przeżycia czasu do remisji choroby u pacjentów leczonych lekiem B](fh_B.pdf){#fig-fhB}

Ponownie jak w przypadku estymatora Kaplana-Meiera, na wykresach [-@fig-fhA] i [-@fig-fhB] widać wyraźne różnice między wykresami estymatorów, co pozwala przypuszczać, że leki A i B nie wykazują podobnego działania.

## Oszacowanie ogona estymatora Kaplana-Meiera

Estymator Kaplana-Meiera jest dobrze określony dla każdego $t \le t_{(n)}$.  
Dodatkowo, jeśli $\delta_{(n)} = 1$, oszacowanie funkcji przeżycia w przedziale $(t_{(n)}, \infty)$ również jest dobrze określone i wynosi $0$.  
Natomiast gdy $\delta_{(n)} = 0$, estymator Kaplana-Meiera nie jest zdefiniowany dla $t > t_{(n)} =: t^+$.  
W takim przypadku Brown, Hollander i Kowar sugerują oszacowanie "ogona" funkcji przeżycia za pomocą odpowiednio dobranej funkcji przeżycia rozkładu wykładniczego, takiej, która w punkcie $t^+$ jest równa $\hat{S}(t^+)$.  

Stąd otrzymujemy:

$$
\hat{S}(t) = \exp \Biggl[ \frac{\ln \hat{S}(t^+)}{t^+} \, t \Biggr], \quad \text{dla } t > t^+
$$

Poniżej przedstawiamy kod w Pythonie generujący oszacowane wartości funkcji przeżycia na ogonie, zgodnie z propozycją Browna, Hollandera i Kowara.

In [9]:
def get_tail(t, df):
    t_plus = np.max(df['Data'])
    theta = np.log(S_hat(t_plus, df)) / t_plus
    return np.exp(theta * t)

Na rysunku [-@fig-kmt] przedstawiamy wykresy estymatora Kaplana-Meiera funkcji przeżycia czasu do remisji choroby u pacjentów leczonych lekami A i B, wraz z "ogonem" oszacowanym zgodnie z propozycją Browna, Hollandera i Kowara.

In [10]:
#| fig-cap: "Wykresy estymatorów Kaplana-Meiera funkcji przeżycia czasu do remisji choroby u pacjentów leczonych lekami A oraz B wraz z ogonem wykładniczym"
#| fig-align: center
#| label: kmt
#| echo: false
def plot_km_with_tail(ax, df):
    time = np.arange(0, np.max(df['Data']), 0.01)
    prob = np.array([S_hat(t, df) for t in time])
    
    tail_start = np.max(df['Data'])
    tail = np.arange(tail_start, 2 + 0.01, 0.01)
    prob_tail = np.array([get_tail(t, df) for t in tail])
    
    x = np.concatenate([time, tail])
    y = np.concatenate([prob, prob_tail])
    
    ax.step(x, y, where='post')
    ax.set_ylim(0, 1.1)
    ax.set_xlabel(r"Czas $t$")
    ax.set_ylabel(r"Estymowana funkcja przeżycia $\hat{S}(t)$")      
fig, axes = plt.subplots(1, 2, figsize=(7, 3))
plot_km_with_tail(axes[0], df_A)
axes[0].set_title("Lek A")
plot_km_with_tail(axes[1], df_B)
axes[1].set_title("Lek B")

plt.tight_layout()
plt.savefig('kmt.pdf')
plt.close()

![Wykresy estymatorów Kaplana-Meiera funkcji przeżycia czasu do remisji choroby u pacjentów leczonych lekami A oraz B wraz z ogonem wykładniczym](kmt.pdf){#fig-kmt}

\newpage

## Symulacja estymatora Kaplana-Meiera dla wygenerowanych danych cenzurowanych

In [11]:
#| echo: false
def GE(n, L, alpha):
    u = np.random.uniform(0, 1, n)
    return -1/L * np.log(1 - u**(1/alpha))
def type_I(t0, L, alpha, n):
    X_nc = GE(n, L, alpha)
    delta = (X_nc <= t0).astype(bool)
    X_c = np.minimum(X_nc, t0)
    return pd.DataFrame({"Data": X_c, "Delta": delta})

W tej sekcji przeprowadzimy symulację wartości estymatora Kaplana-Meiera z "ogonem" oszacowanym zgodnie z propozycją Browna, Hollandera i Kowara. Symulacja dotyczy wartości w punktach $t_0$ oraz $2t_0$, przy czym za parametr $t_0$ przyjęliśmy w przybliżeniu wartość oczekiwaną badanego rozkładu, tj. $t_0 = 1.5$.

Symulacja oparta będzie na danych cenzurowanych I-go typu, generowanych z rozkładu $\mathcal{GE}(\lambda = 1, \alpha = 2)$. Wygenerujemy $M = 1000$ niezależnych zbiorów danych dla trzech rozmiarów próby: $n \in \{30, 50, 100\}$.

Poniżej przedstawiamy kod symulacji w języku Python.

In [12]:
def sim(M=1000, alpha=2, L=1, n_values=[30, 50, 100]):
    t0=1.5
    results = {}
    np.random.seed(38)
    for n in n_values:
        est_t0 = []
        est_2t0 = []    
        for _ in range(M):
            df = type_I(t0=t0, L=L, alpha=alpha, n=n)
            t_plus = np.max(df['Data'])
            if t0 <= t_plus:
                S_t0 = S_hat(t0, df)
            else:
                S_t0 = get_tail(t0, df)
            if 2*t0 <= t_plus:
                S_2t0 = S_hat(2*t0, df)
            else:
                S_2t0 = get_tail(2*t0, df)
            est_t0.append(S_t0)
            est_2t0.append(S_2t0)
        results[n] = {
            't0': np.array(est_t0),
            '2t0': np.array(est_2t0)
        }
    return results

In [13]:
#| fig-cap: 'Histogramy wartości estymatora Kaplana-Meiera z "ogonem" w punktach $t_0$ i $2t_0$, uzyskane w wyniku symulacji dla $M = 1000$ niezależnych zbiorów danych cenzurowanych I-go typu z rozkładu $\text{GE}(\lambda = 1, \alpha = 2)$ dla różnych rozmiarów próby $n \in \{30, 50, 100\}$'
#| fig-align: center
#| label: sim
#| echo: false
results = sim()

fig, axes = plt.subplots(2, 3, figsize=(22, 16))
n_list = sorted(results.keys())

for i, n in enumerate(n_list):
    axes[0, i].hist(results[n]['t0'], bins=23, color='skyblue', edgecolor='black')
    axes[0, i].set_title(rf'Estymacje $\hat{{S}}(t_0)$, n={n}', fontsize=30)
    axes[0, i].set_xlabel(r'$\hat{S}(t_0)$', fontsize=25)
    axes[0, i].set_ylabel('Liczebność', fontsize=25)
    axes[0, i].tick_params(axis='x', labelsize=25)
    axes[0, i].tick_params(axis='y', labelsize=25)

for i, n in enumerate(n_list):
    axes[1, i].hist(results[n]['2t0'], bins=23, color='salmon', edgecolor='black')
    axes[1, i].set_title(rf'Estymacje $\hat{{S}}(2 t_0)$, n={n}', fontsize=30)
    axes[1, i].set_xlabel(r'$\hat{S}(2 t_0)$', fontsize=25)
    axes[1, i].set_ylabel('Liczebność', fontsize=25)
    axes[1, i].tick_params(axis='x', labelsize=25)
    axes[1, i].tick_params(axis='y', labelsize=25)
    
plt.tight_layout()
plt.savefig('sim.pdf')
plt.close()

![Histogramy wartości estymatora Kaplana-Meiera z "ogonem" w punktach $t_0$ i $2t_0$, uzyskane w wyniku symulacji dla $M = 1000$ niezależnych zbiorów danych cenzurowanych I-go typu z rozkładu $\text{GE}(\lambda = 1, \alpha = 2)$ dla różnych rozmiarów próby $n \in \{30, 50, 100\}$](sim.pdf){#fig-sim}

\newpage

Na wykresie [-@fig-sim] przedstawiamy histogramy oszacowań estymatora Kaplana-Meiera z ogonem w punktach $t_0$ oraz $2t_0$ dla różnych rozmiarów próby $n$. 

Na podstawie przedstawionych histogramów widać, że wraz ze wzrostem rozmiaru próby $n$ kształty rozkładu oszacowań estymatora Kaplana-Meiera stają się coraz mniej podobne do kształtu rozkładu normalnego. Obserwacja ta nie pozwala na wyciąganie wniosków o jego asymptotycznej normalności w punktach $t_0$ oraz $2t_0$.

# Estymacja średniego czasu życia

Niech $\mu = \mathbb{E}[X]$ oznacza średni czas do wystąpienia zdarzenia.
Jeżeli zmienna losowa $X$ ma absolutnie ciągły rozkład o nośniku $(0,\infty)$
oraz funkcję przeżycia $S(t)$, to wartość oczekiwana może zostać zapisana jako

$$
\mu = \int_{0}^{\infty} S(t)\, dt
$$

Aby oszacować średni czas życia, zastępujemy nieznaną funkcję przeżycia $S(t)$ jej estymatorem.
W dalszej części wykorzystujemy w tym celu estymator Kaplana-Meiera oraz estymator Fleminga-Harringtona.

W praktyce często zdarza się, że obserwacja $t_{(n)}$ jest cenzurowana.
W takim przypadku rozważane estymatory $S(t)$ nie są określone dla $t>t_{(n)}$, co uniemożliwia bezpośrednie zastosowanie wzoru na wartość oczekiwaną. Aby oszacować ogon całki, niezbędne jest przedłużenie estymowanej funkcji przeżycia.

W tym celu stosujemy podejście Browna, Hollandera i Kowara,
polegające na dopasowaniu wykładniczego ogona tak, aby pokrywał się on
z estymowaną funkcją przeżycia w ostatnim niecenzurowanym punkcie $t_{\max}$.

Całkując tą aproksymację otrzymujemy

$$
\int_{t_{\max}}^{\infty} S(t)\, dt
= \int_{t_{\max}}^{\infty} 
  \exp\!\left( \frac{\ln S(t_{\max})}{t_{\max}}\, t \right) dt
= -\,\frac{t_{\max}}{\ln S(t_{\max})}\, S(t_{\max})
$$

Wyrażenie to stanowi estymację brakującej części całki, którą należy dodać do części policzonej na przedziale $[0, t_{\max}]$.

Poniżej przedstawiamy implementację opisanego estymatora dla rozważanych funkcji przeżycia, z możliwością uwzględnienia ogona, w języku Python.

In [14]:
def est_mean(df, est="KM", wtail=True):
    if est == "KM":
        S_func = S_hat
    elif est == "FH":
        S_func = S_tld
    else:
        raise ValueError("Nieznany typ estymatora")
    data = df['Data'].values
    delta = df['Delta'].values
    sort_idx = np.argsort(data)
    data = data[sort_idx]
    delta = delta[sort_idx]
    t_max = np.max(data)
    times = np.concatenate([[0], data])
    times = times[times <= t_max]
    surv_prob = np.array([S_func(t, df) for t in times])
    mean_t = 0.0
    for i in range(len(times) - 1):
        dt = times[i + 1] - times[i]
        area = surv_prob[i] * dt
        mean_t += area
    if wtail:
        S_tmax = S_func(t_max, df)
        if S_tmax > 0:
            tail_integral = -t_max / np.log(S_tmax) * S_tmax
            mean_t += tail_integral
    return mean_t

In [15]:
#| echo: false
#| label: tbl-mean-est
#| tbl-cap: Średnie czasy przeżycia dla Leku A i Leku B oszacowane przy użyciu estymatorów Kaplana-Meiera i Fleminga-Harringtona
leki = ["A", "B"]
estymatory = ["KM", "FH"]
ogon_opts = [False, True]

records = []

for lek in leki:
    df = df_A if lek == "A" else df_B
    for est in estymatory:
        for wtail in ogon_opts:
            mean_val = est_mean(df, est, wtail)
            records.append({
                "Lek": lek,
                "Estymator": est,
                "Ogon": "tak" if wtail else "nie",
                "Średnia": mean_val
            })

params_df = pd.DataFrame(records)

table_latex = tabulate(
    params_df,
    headers='keys',
    tablefmt='latex_booktab',
    showindex=False,
    floatfmt=".4f"
)
Markdown(table_latex)

Lek    Estymator    Ogon      Średnia
-----  -----------  ------  ---------
A      KM           nie        0.7108
A      KM           tak        1.4321
A      FH           nie        0.7179
A      FH           tak        1.4840
B      KM           nie        0.6729
B      KM           tak        1.3943
B      FH           nie        0.6810
B      FH           tak        1.4471

W tabeli [-@tbl-mean-est] przedstawiamy średnie czasy do remisji choroby pacjentów leczonych lekiem A i lekiem B, oszacowane za pomocą powyższej metody. Uwzględniamy estymatory Kaplana-Meiera oraz Fleminga-Harringtona, zarówno w wersjach z oszacowaniem ogona, jak i bez niego.

Można zauważyć, że w przypadku wersji bez szacowania ogona oszacowane średnie czasy za pomocą estymatorów Kaplana-Meiera oraz Fleminga-Harringtona są bardzo zbliżone. Natomiast w wersjach z oszacowaniem ogona różnice między średnimi stają się bardziej widoczne. Zgodnie z intuicją oraz na podstawie wcześniejszych wyników dotyczących leków można dojść do wniosku, że pominięcie szacowania ogona jest błędne. Rzeczywiście, dane cenzurowane nie zakładają, że zdarzenie nie występuje po momencie cenzury, zatem estymacja średniej wyłącznie na podstawie danych niecenzurowanych daje niedokładną wartość.

Zauważmy również, że istotne jest rozróżnienie pomiędzy samymi estymatorami. Estymator Kaplana-Meiera daje wartości nieco mniejsze niż estymator Fleminga-Harringtona.

# Estymacja przedziałowa wartości średniego czasu życia

Do wyznaczenia asymptotycznego punktowowo przedziału ufności wykorzystamy asymptotyczne własności estymatora
$$
\hat{\mu}_\tau = \int_0^\tau \hat{S}(t)\, dt,
$$
gdzie $\tau$ jest przyjętym czasem odzwierciedlającym wiedzę a priori o maksymalnym czasie do wystąpienia zdarzenia.

Funkcję przeżycia $S$ szacujemy na przedziale $(t_{(n)}, \tau)$ poprzez $\hat{S}(t_{(n)})$.

Okazuje się, że przy odpowiednich założeniach estymator ten jest asymptotycznie normalny
$$
\hat{\mu}_\tau \sim \mathcal{AN}\bigl(\mu_\tau,\; V(\hat{\mu}_\tau)\bigr),
$$
gdzie $V(\hat{\mu}_\tau)$ oznacza wariancję estymatora $\hat{\mu}_\tau$.

Wariancja ta jest zazwyczaj nieznana, dlatego musi zostać oszacowana. Za estymator wariancji estymatora $\hat{\mu}_\tau$ przyjmujemy
$$
\hat{V}(\hat{\mu}_\tau)
=
\sum_{i=1}^D
\left(
\int_{s_i}^{\tau} \hat{S}(t)\, dt
\right)^2
\frac{d_i}{r_i(r_i - d_i)},
$$
gdzie $r_i$ oraz $d_i$ są zdefiniowane jak wcześniej, natomiast $D$ oznacza liczbę różnych, uporządkowanych, niecenzurowanych czasów zdarzeń $s_i$.

Wtedy przedział postaci $[T_L, T_U]$, gdzie
$$
T_L = \hat{\mu}_\tau - z(1 - \alpha/2)\,\sqrt{\hat{V}(\hat{\mu}_\tau)},
$$
$$
T_U = \hat{\mu}_\tau + z(1 - \alpha/2)\,\sqrt{\hat{V}(\hat{\mu}_\tau)},
$$
a $z(q)$ oznacza kwantyl rzędu $q$ standardowego rozkładu normalnego, stanowi asymptotyczny punktowy przedział ufności na poziomie istotności
$1 - \alpha$ dla wartości średniej $\mu$ rozkładu czasu życia.

Poniżej przedstawiamy kod funkcji obliczającej wartości całki z funkcji $\hat{S}(t)$ na zadanym przedziale oraz funkcji zwracającej realizację przedziału ufności na wybranym poziomie istotności.

In [16]:
def int_s(df, tau, s=0, est="KM"):
    if est == "KM":
        S_func = S_hat
    elif est == "FH":
        S_func = S_tld
    else:
        raise ValueError("Nieznany typ estymatora")
        
    data = df["Data"].values
    times = data[(data >= s) & (data <= tau)]
    times = np.concatenate([[s], times, [tau]])
    times = np.sort(times)
    surv_vals = np.array([S_func(t, df) for t in times])
    
    integral = 0.0
    for i in range(len(times) - 1):
        dt = times[i+1] - times[i]
        integral += surv_vals[i] * dt
        
    return integral
    
def mu_estimate(df, tau, alpha=0.05):
    data = df["Data"].values
    delta = df["Delta"].values

    sort_idx = np.argsort(data)
    data = data[sort_idx]
    delta = delta[sort_idx]

    mu_tau = int_s(df, tau)
    event_times = np.unique(data[delta == 1])

    var = 0.0
    for s_i in event_times:
        if s_i > tau:
            break
        r_i = np.sum(data >= s_i)
        d_i = np.sum((data == s_i) & (delta == 1))

        S = int_s(df, tau, s=s_i)

        var += (S ** 2) * d_i / (r_i * (r_i - d_i))

    V = np.sqrt(var)
    z = norm.ppf(1 - alpha/2)
    lower = mu_tau - z*V
    upper = mu_tau + z*V

    return lower, upper

Rozważamy dane pochodzące z badania przeprowadzonego w Mayo Clinic, obejmującego pacjentki z rakiem jajnika w stadium II lub IIIA. Głównym celem jest ustalenie, czy stopień zaawansowania choroby był związany z czasem do progresji choroby. W tym celu wyznaczymy realizacje przedziałów ufności dla średniego czasu do progresji choroby w dwóch badanych grupach pacjentek.

Przyjmujemy poziom istotności 
$$
\alpha = 0.05
$$
oraz maksymalny czas do wystąpienia zdarzenia 
$$
\tau_1 = 760 \quad \text{oraz} \quad \tau_2 = 985
$$

Wartości $\tau$ obliczyliśmy korzystając w obu grupach ze wzoru:
$$
\tau = t_{\max}^{\text{niecenzurowany}} + (1 - p_c) \cdot \left( t_{\max}^{\text{cenzurowany}} - t_{\max}^{\text{niecenzurowany}} \right),
$$
gdzie:

- $t_{\max}^{\text{niecenzurowany}}$ to ostatni moment, w którym odnotowano progresję choroby,
  
- $t_{\max}^{\text{cenzurowany}}$ to największy czas obserwacji w grupie,

-  $p_c$ oznacza proporcję obserwacji cenzurowanych w grupie.

Wzór uwzględnia intuicje, że im większa jest liczba danych cenzurowanych, tym mniejsza powinna być szansa na wystąpienie dalszych zdarzeń niecenzurowanych.

Dla tak wybranych wartości $\alpha$ i $\tau$ otrzymujemy następujące realizację przedziałów ufności:

In [17]:
full_A = np.array([28, 89, 175, 195, 309, 462])
censored_A = np.array([377, 393, 421, 447, 709, 744, 770, 1106, 1206])
data_A = np.concatenate([full_A, censored_A])
delta_A = np.array([True]*len(full_A) + [False]*len(censored_A))
 
full_B = np.array([34, 88, 137, 199, 280, 291, 309, 351, 358, 369,
                   369, 370, 375, 382, 392, 451])
censored_B = np.array([299, 300, 429, 1119])
data_B = np.concatenate([full_B, censored_B])
delta_B = np.array([True]*len(full_B) + [False]*len(censored_B))
 
df_A = pd.DataFrame({"Data": data_A, "Delta": delta_A})
df_B = pd.DataFrame({"Data": data_B, "Delta": delta_B})
tau_1 = 760
tau_2 = 985
L_A_1, R_A_1  = mu_estimate(df_A, tau_1)
L_B_2, R_B_2 = mu_estimate(df_B, tau_2)
L_A_3, R_A_3  = mu_estimate(df_A, tau_2)
L_B_4, R_B_4  = mu_estimate(df_B, tau_1)

In [18]:
#| echo: false
#| label: tbl-tau-ci
#| tbl-cap: Realizacje przedziałów ufności funkcji przeżycia dla obu grup pacjentek przy różnych wartościach tau

rows = [
    {"Tau": 760, "Grupa": "niski stopień", "Przedział ufności": f"[{L_A_1:.2f}, {R_A_1:.2f}]"},
    {"Tau": 760, "Grupa": "wysoki stopień", "Przedział ufności": f"[{L_B_4:.2f}, {R_B_4:.2f}]"},
    {"Tau": 985, "Grupa": "niski stopień", "Przedział ufności": f"[{L_A_3:.2f}, {R_A_3:.2f}]"},
    {"Tau": 985, "Grupa": "wysoki stopień", "Przedział ufności": f"[{L_B_2:.2f}, {R_B_2:.2f}]"}
]

table_latex = tabulate(
    rows,
    headers="keys",
    tablefmt="latex_booktab",
    showindex=False
)

Markdown(table_latex)

  Tau  Grupa           Przedział ufności
-----  --------------  -------------------
  760  niski stopień   [379.71, 673.54]
  760  wysoki stopień  [269.24, 427.67]
  985  niski stopień   [445.11, 858.13]
  985  wysoki stopień  [260.25, 476.04]

Wyniki przedstawione w tabeli [-@tbl-tau-ci] pokazują, że dla obu wartości $\tau$ zarówno górne, jak i dolne granice przedziałów ufności są niższe w przypadku pacjentek z wysokim stopniem zaawansowania choroby niż u pacjentek z niższym stopniem zaawansowania. Ponadto, przedziały te są węższe dla grupy o wysokim stopniu. Porównując wartości $\tau$, można zauważyć, że ma on istotny wpływ w grupie o niskim stopniu zaawansowania, gdzie przedziały zostały przesunięte, natomiast w grupie o wysokim stopniu jego wpływ jest znikomy.

# Weryfikacja hipotezy o jednakowym rozkładzie czasów życia

In [19]:
#| echo: false
df = np.concatenate([data_A, data_B])
df_uc = np.unique(np.concatenate([full_A, full_B]))
df_uc = np.sort(df_uc)
d = pd.DataFrame({
    'Data': np.concatenate([data_A, data_B]),
    'Delta': np.concatenate([delta_A, delta_B]),
    'Groups': np.concatenate([np.zeros(len(data_A)), np.ones(len(df) - len(data_A))])
})

Celem tej części analizy jest porównanie czasu do wystąpienia progresji choroby w dwóch badanych grupach pacjentek. Przyjmujemy poziom istotności $\alpha = 0.05$. Weryfikowana jest hipoteza zerowa o równości funkcji przeżycia w obu grupach:

$$
H_0: S_1(t) = S_2(t) \quad \text{(rozkłady są jednakowe)},
$$
$$
H_1: S_1(t) \neq S_2(t) \quad \text{(rozkłady różnią się)}
$$

Statystyka testowa stosowana do weryfikacji hipotezy oparta jest na wektorze $k-1$ statystyk $Z_j(\tau)$ i ma postać:

$$
Z^2 = \mathbf{Z}(\tau)\, \Sigma^{-1}\, \mathbf{Z}(\tau)^{T}
$$

gdzie:

- $\Sigma$ jest macierzą oszacowanych wariancji $\hat{\sigma}_{jj}$ oraz kowariancji $\hat{\sigma}_{ij}$ statystyk $Z_j(\tau)$,

- $\mathbf{Z}(\tau) = (Z_1(\tau), \ldots, Z_{k-1}(\tau))$

Statystyka $Z^2$ dąży asymptotycznie w rozkładzie do rozkładu $\chi^2$ z $k-1$ stopniami swobody.

W naszym przypadku rozważamy dwie grupy pacjentek, zatem $k = 2$. Statystyka $Z_j(\tau)$ ma postać:
$$
Z_j(\tau) = \sum_{i=1}^{D} W(t_i)\left( d_{ij} \;-\; r_{ij}\frac{d_i}{r_i} \right)
$$

gdzie stosujemy oznaczenia:

- $D$ - liczba zdarzeń niecenzurowanych w całej próbie,

- $d_{ij}$ - liczba zdarzeń w grupie $j$ w chwili $t_i$,

- $r_{ij}$ - liczba pacjentek w grupie $j$ w stanie ryzyka w chwili $t_i$,

- $d_i = \sum d_{ij}$

- $r_i = \sum r_{ij}$

- $W(t_i)$ - funkcja wagowa charakterystyczna dla wybranego testu

W analizie porównujemy cztery warianty testu, różniące się doborem wag $W(t_i)$:

- test log-rank,
  
- test Gehana-Breslowa,

- test Tarone'a-Warego,

- Test Peto-Peto.


Wartości $p$ obliczono przy użyciu pakietu `lifelines.` Wyniki analizy przedstawiamy w Tabeli [-@tbl-pval-tests].

In [20]:
def pval (test, df):
    if test == "log-rank":
        result = multivariate_logrank_test(df['Data'], df['Groups'],
                                df['Delta'])
    elif test == "Gehan-Breslow":
        result = multivariate_logrank_test(df['Data'], df['Groups'],
                                df['Delta'], weightings="wilcoxon")
    elif test == "Tarone-Ware":
        result = multivariate_logrank_test(df['Data'], df['Groups'],
                                df['Delta'], weightings="tarone-ware")
    elif test == "Peto-Peto":
        result = multivariate_logrank_test(df['Data'], df['Groups'],
                                df['Delta'], weightings="peto")
    return result

In [21]:
#| echo: false
#| label: tbl-pval-tests
#| tbl-cap: Wartości p dla czterech analizowanych testów
tests = ["log-rank", "Gehan-Breslow", "Tarone-Ware", "Peto-Peto"]
p_values = [pval(test, d).p_value for test in tests]
params_df = [{"Test": test, "p-value": pv} for test, pv in zip(tests, p_values)]

table_latex = tabulate(
    params_df,
    headers='keys',
    tablefmt='latex_booktab',
    showindex=False,
    floatfmt=".4f"
)
Markdown(table_latex)

Test             p-value
-------------  ---------
log-rank          0.0183
Gehan-Breslow     0.1342
Tarone-Ware       0.0550
Peto-Peto         0.1015

Wyniki zastosowanych testów wykazują wyraźną rozbieżność. Test log-rank prowadzi do odrzucenia hipotezy zerowej, podczas gdy pozostałe testy nie dają do tego podstaw. Dodatkowo otrzymane wartości $p$ znacznie się różnią: testy Gehana-Breslowa oraz Peto-Peto dają wysokie wartości $p$, znacznie przekraczające poziom istotności $\alpha$, test Tarone'a-Warego znajduje się blisko granicy istotności, natomiast test log-rank zwraca wartość zdecydowanie niższą niż $\alpha$. Taka rozbieżność uniemożliwia jednoznaczne rozstrzygnięcie, czy hipotezę zerową należy odrzucić.

Różnice te mogą wynikać z odmiennych własności danych, na które poszczególne testy kładą nacisk poprzez zastosowane funkcje wagowe. Aby to zweryfikować, przeanalizujemy krzywe przeżycia dla obu grup oraz funkcje wagowe poszczególnych testów, przedstawione odpowiednio na rysunkach [-@fig-km_prog] i [-@fig-weight_f]. Poniżej przedstawiamy też implementację używanych funkcji wagowych.

\newpage

In [22]:
def W_t(test, df, df_uc):
    df_uc = np.asarray(df_uc)
    D = len(df_uc)
    r_i = np.array([np.sum(df_uc >= t) for t in df_uc])
    if test == "log-rank":
        W = np.ones(D)
    elif test == "Gehan-Breslow":
        W = r_i
    elif test == "Tarone-Ware":
        W = np.sqrt(r_i)
    elif test == "Peto-Peto":
        data = df['Data'].to_numpy()
        delta = df['Delta'].to_numpy()
        order = np.argsort(data)
        data = data[order]
        delta = delta[order]
        W = []
        for t in df_uc:
            W_ti = 1.0
            for j in range(len(data)):
                if data[j] > t:
                    break
                r_j = np.sum(data >= data[j])
                d_j = np.sum((data == data[j]) & (delta == 1))
                W_ti *= (1 - d_j / (r_j+1))

            W.append(W_ti)
        W = np.array(W)
    return W / np.sum(W)

In [23]:
#| fig-cap: "Wykres estymatora Kaplana-Meiera funkcji przeżycia czasu do progresji choroby w obu grupach"
#| fig-align: center
#| label: km_prog
#| echo: false

time = np.linspace(0, 800, 200)

S_A = [S_hat(t, df_A) for t in time]
S_B = [S_hat(t, df_B) for t in time]

plt.figure(figsize=(5, 3))

plt.step(time, S_A, where="post", label="Grupa A")
plt.step(time, S_B, where="post", label="Grupa B")

plt.xlabel(r"Czas $t$")
plt.ylabel(r"Estymowana funkcja przeżycia $\widehat{S}(t)$")
plt.title("Estymator Kaplana-Meiera w obu grupach pacjentek")
plt.legend()

plt.tight_layout()
plt.savefig("km_prog.pdf")
plt.close()

![Wykres estymatora Kaplana-Meiera funkcji przeżycia czasu do progresji choroby w obu grupach](km_prog.pdf){#fig-km_prog}

In [24]:
#| fig-cap: "Wizualizacja funkcji wagowej $W(t)$ dla czterech badanych testów"
#| fig-align: center
#| label: weight_f
#| echo: false

w_logrank = W_t("log-rank", d, df_uc)
w_gehan = W_t("Gehan-Breslow", d, df_uc)
w_tarone = W_t("Tarone-Ware", d, df_uc)
w_peto = W_t("Peto-Peto", d, df_uc)

plt.figure(figsize=(6, 4))

plt.plot(df_uc, w_logrank, label='log-rank', linewidth=2)
plt.plot(df_uc, w_gehan, label='Gehan-Breslow', linewidth=2)
plt.plot(df_uc, w_tarone, label='Tarone-Ware', linewidth=2)
plt.plot(df_uc, w_peto, label='Peto-Peto', linewidth=2)

plt.xlabel(r"Czas $t$")
plt.ylabel(r"Waga $W(t)$")
plt.title("Funkcje wagowe wybranych testów")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig("weight_functions.pdf")
plt.close()

![Wizualizacja funkcji wagowej $W(t)$ dla czterech badanych testów](weight_functions.pdf){#fig-weight_f}

\newpage

Na wykresie [-@fig-km_prog] możemy zauważyć, że funkcje przeżycia obu grup zachowują się bardzo podobnie dla małych wartości $t$, natomiast dla większych wartości czasu pojawiają się wyraźne różnice. W konsekwencji testy, które kładą większy nacisk na zdarzenia w późniejszym okresie obserwacji, są w stanie wykryć rozbieżności w rozkładach, podczas gdy testy skupiające się na początkowych zdarzeniach tego nie zrobią.

Analiza wykresu funkcji wagowych [-@fig-weight_f] potwierdza tę obserwację. Test log-rank stosuje równomierne wagi w całym okresie obserwacji, co pozwala mu wychwycić różnicę w rozkładach obu grup i skutkuje niską wartością $p$. Natomiast test Gehana-Breslowa mocno koncentruje się na początkowych zdarzeniach, a wagi dla późniejszych czasów są niewielkie, co tłumaczy wysoką wartość $p$ tego testu.

Jeśli chodzi o testy Peto-Peto i Tarone'a-Warego, ich funkcje wagowe są podobne dla końcowych zdarzeń, ale różnią się na początku obserwacji. W efekcie test Tarone'a-Warego jest bardziej czuły na różnice w rozkładach niż test Peto-Peto, co sprawia, że jego wartość $p$ znajduje się bliżej poziomu istotności.